# Computer vision - Laboratory

==============================================================================================
## Practicum 3: Image and Video Segmentation

==============================================================================================

The main topics are:

1)	Segmentation of video shots with static scenes.

2) Background substraction.

3)	Segmentation of images.

In order to successfuly complete this practicum it is necessary to understand the following theory concepts: background substraction, K-means clustering, etc.

The following chapters of the book “Computer Vision: Algorithms and Applicatons” from Richard Szeliski have further information about the topic:

* Chapter 4: Computer Vision: Algorithms and Applications.

* Chapter 5: Segmentation.

## 3.1 Background substraction methods


Given the video stored in ‘Barcelona-sequence’, which contains images acquired by a static camera, remove all the "artifacts" considered as foreground related to movement extracting the background images.

Note: One of the applications of these methods is the button "remove tourists" implemented in most commercial photo cameras. For instance, Adobe uses the "Monument Mode", which automatically deletes the people going by the cameras.

Read and visualize the sequence of images "images/Barcelona-sequence"
Hint: In order to read a  collection of images, we wil use the function animation.FuncAnimation [https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.animation.FuncAnimation.html].

Observe in the following example, how FuncAnimation is used to read and visualize a sequence of frames. Explore the parameters of animation.FuncAnimation().

In [ ]:
import numpy as np
import skimage
from skimage import io
import matplotlib.pyplot as plt

ic = io.ImageCollection('images/Barcelona-sequence/*.png')
        # Reading a sequence of images from a folder

import matplotlib.animation as animation
%matplotlib nbagg 
    #Changing the pluggin is necessary always when visualizing a video!

i=0       #Inicializing the video display
fig = plt.figure()  # Create figure
im = plt.imshow(ic[i], animated=True) #Visualize the first image

def updatefig1(i):   #Updating the frame visualization
    im.set_array(ic[i*5]) #Changing the content of the canvas
    return im, #to return a tuple!

ani = animation.FuncAnimation(fig, updatefig1, interval=5, blit=False, frames=50, repeat= False)

a) Find where a shot (scene) finishes and the following starts (boundaries). Each of the scenes in a video is usually called 'shot'. Which measure can be used in order to visually distinguish the shots in a plot? Explain your solution.

Show the initial and final images of each shot extracted as follows:

<img src="images_for_notebook/result_shot_detection.png">

**Hint:** take the following example of video and temporal plot visualization as a template. The plot must be replaced by a frame by frame measure to be defined by you, being applicable to distinguish the shots.

- If you need to convert the image to float, the command is: img_as_float()
- If you need the histogram, it is in skimage.exposure

In [ ]:
ic_gray = [skimage.color.rgb2gray(frame) for frame in ic]

histoarray = [np.histogram(frame) for frame in ic_gray]

In [ ]:
means = []
stds = []

for i in range(len(histoarray)):
    curr = histoarray[i]
    means.append(np.mean(curr[0]))
    stds.append(np.std(curr[0]))

In [ ]:
cosas = []

# We add to our histogram array the values letting them between 0 and 1
for i in range(len(histoarray)):
    y = means[i] + stds[i]
    cosas.append(y/10000)

In [ ]:
# Example
# Sinusoidal plot points generation
def data_gen():
    t = data_gen.t
    cnt = 0
    
    while cnt < len(ic):
        cnt+=1
        t += 0.05
        y = abs(cosas[cnt]-cosas[cnt-1])
        # adapted the data generator to yield both sin and cos
        yield t, y

data_gen.t = 0

%matplotlib nbagg

# create a figure with two subplots
fig, (ax1, ax2) = plt.subplots(2,1)

# intialize a line object on the second axes for plotting
line, = ax2.plot([], [], lw=2, color='r')

ax2.set_ylim(0, 2)
ax2.set_xlim(0, 5)
ax2.grid()

# initialize the data arrays 
xdata, ydata = [], []
def run(data):
    # update the data plot
    t, y = data
    xdata.append(t) # time = x axis
    ydata.append(y) # y axis

    # Plot image on top row
    ax1.imshow(ic[len(xdata)])
          
    # Plot sin in bottom row
    xmin, xmax = ax2.get_xlim()
    if t >= xmax:
        ax2.set_xlim(xmin, 2*xmax)
        ax2.figure.canvas.draw()
            
    # update the data of both line objects
    line.set_data(xdata, ydata)

    return line

ani = animation.FuncAnimation(fig, run, data_gen, blit=True, interval=10, repeat=False)
plt.show()

In [ ]:
# We crete a function that displays 2 images with their respective titles
def print_1x2(imatge,imatge_despres):
    fig, axarr = plt.subplots(1,2)
    fig.set_size_inches(25,10)
    
    #imatge
    axarr[0].imshow(imatge,cmap='gray')
    axarr[0].set_title('Inici de seqüència')
    axarr[0].xaxis.set_visible(False)
    axarr[0].yaxis.set_visible(False)
    
    #imatge després
    axarr[1].imshow(imatge_despres, cmap='gray')
    axarr[1].set_title('Final de seqüència')
    axarr[1].xaxis.set_visible(False)
    axarr[1].yaxis.set_visible(False)
    plt.show()

%matplotlib inline

hard_jump = []
thr = 0.71

last_idx = 0

# We check where the sequences end defining a threshold 'thr' defined by our graph
for i in range(1,len(cosas)):
    if(abs(cosas[i-1]-cosas[i]) >= thr):
        print_1x2(ic[last_idx],ic[i-1])
        hard_jump.append((last_idx,i))
        last_idx = i+1

# We must check if there is a sequence between the last_idx found and the last image

if(last_idx != len(ic)):    
    print_1x2(ic[last_idx], ic[-1])
    hard_jump.append((last_idx, len(ic)))

## 3.2 Background substraction

Apply the background substraction algorithm (check theory material).

Visualize, for each shot of the video:
    1) images belonging to the shot
    2) the background image, and
    3) the foreground.
    
**Hint**: You can construct a mask obtained from the original image and the background in order to know which parts of the image form part from the foreground and recover from the original image just the foreground regions.

In [ ]:
from skimage import img_as_float

def print_1x1(imatge,title):
    fig, axarr = plt.subplots(1,1)
    fig.set_size_inches(10,10)
    
    #image
    axarr.imshow(imatge,cmap='gray')
    axarr.set_title(title)
    axarr.xaxis.set_visible(False)
    axarr.yaxis.set_visible(False)

    plt.show()

def find_background(start,end):
    
    # We define an image with the same shape as the original ones just to fill it later
    background_image = np.zeros([ic[0].shape[0],ic[0].shape[1], ic[0].shape[2]], np.uint8)
    
    # We substract the sequence we want to extract the background
    sequence = ic[start:end]
    
    # We need
    red_channel_arr = []
    green_channel_arr = []
    blue_channel_arr = []
    
    for img in sequence:
        red_channel_arr.append(img_as_float(img[:,:,0]))
        green_channel_arr.append(img_as_float(img[:,:,1]))
        blue_channel_arr.append(img_as_float(img[:,:,2]))
        
    #we calculate the median for every channel, using every image in shot_images
    red_channel_median = np.median(red_channel_arr, axis = 0)       
    green_channel_median = np.median(green_channel_arr, axis = 0)       
    blue_channel_median = np.median(blue_channel_arr, axis = 0)       
    
    for i in range(sequence[0].shape[0]):
        for j in range(sequence[0].shape[1]):
            background_image[i,j] = [red_channel_median[i,j]*255,green_channel_median[i,j]*255,blue_channel_median[i,j]*255]

    return background_image

idx = 1
for pair in hard_jump:
    print_1x1(find_background(pair[0],pair[1]), "Background sequence "+str(idx))
    idx+=1

Comment your implementation including details. What happens if the shots are not correctly extracted? What happens if you find too many shots in the video? What do the static background images represent? In which situations does the algorithm work and in which it does not? What happens if you substract the background image from the original one?

Do you see any additional application for this algorithm?

**[OPTIONAL]**
Apply the algorithm to some other static video that you found.

The implementation is commented inside itself.

If the shots are not correctly extracted, the background image will be computed along the out-shot images which will end up being worse than the correct one (depending on how many out-shot images from all of them we take).



In [ ]:
from skimage.filters import threshold_mean
from skimage import img_as_float

%matplotlib inline

def print_1x1(imatge,title):
    fig, axarr = plt.subplots(1,1)
    fig.set_size_inches(10,10)
    
    #imatge
    axarr.imshow(imatge,cmap='gray')
    axarr.set_title(title)
    axarr.xaxis.set_visible(False)
    axarr.yaxis.set_visible(False)

    plt.show()


extraction = ic[0] - threshold_mean(find_background(hard_jump[0][0],hard_jump[0][1]))
print_1x1((extraction*255).astype('uint8'), "Background extraction")

## 3.3 Clustering methods on the RGB-XY space

a) Read any image from the folder 'images' and segment it using Felzenszwalbs's method. Test different parameters in order to obtain a good image segmentation. How does each parameter behave? Which are the optimal values? Comment what algorithm is the method based in up to 3 lines most.

**Hint**: 
- Different image segmentation commands can be found in skimage.segmentation.
- Use the function segmentation.mark_boundaries for seeing the boundaries of the segments.
- Use the inline pluggin to visualize images (%matplotlib inline)
- Add title to the figures to explain what is displayed.

b) Segment the previous image using SLIC algorithm. Test different parameters in order to obtain a good image segmentation. How does each parameter behave? Upt o your opinion, which are the optimal values? Comment what algorithm is the method based in up to 3 lines.

c) Plot the original image and the results of both algorithms in a 1x3 subplot. Calculate also the number of segments obtained on the image by each of the algorithms. Comment the differences between each method as well as their advantages and disadvantages.